# Aufgabe 1 (5P)

## a)
Nutzen Sie vom Datensatz „pvtest.csv“ die Werte 'Edaily','Dci','Dcp','Dcu','Temp1','hour‘ und erstellen Sie einen DataFrame
als Zeitreihe, ähnlich wie unten abgebildet. Auf Grundlage dieser Daten soll der Energieertrag Dcp immer in 5 Minuten in
der Zukunft prognostiziert werden. (0.5P)

<img src='images/1a.png' width=400>

## b)
Teilen Sie die Daten sinnvoll in Lern-und Testdatensatz auf, so dass die zeitlichen Reihenfolgen bestehen bleiben ! (0.5P)

## c)
Standardisieren Sie die Daten mit dem Standardscaler und restrukturieren Sie die Zeitreihe in einzelne Datenfenster, so dass
Sie aus allen vorhandenen 6 Features aus den letzten drei Stunden immer jeweils den Energieertrag für die nächsten 5
Minuten vorhersagen können. (1,5P)

# d)
Definieren und kompilieren Sie jetzt ein passendes Modell in Keras, welches mind. eine LSTM-Layer nutzt und DropoutLayer einfügt, um „Überanpassung“ zu verhindern. (1P)

## e)
Trainieren Sie das Modell mit allen Lerndaten über mind. 100 Epochen, nutzen Sie ggf. auch callbacks, um den Lernprozess
zu verkürzen. Verbessern Sie Ihr Training und Ihre Modellarchitektur so lange, bis Sie einen Mean absolute Error von mind.
50 erzielen! (1P)

# f)
Visualisieren Sie Ihre Prognoseergebnisse, so ähnlich wie unten angegeben im Vergleich gemessener Daten mit
prognostizierten Werten! (0.5P)

<img src="images/1f.png" width=900>